#Податочно множество
Симнете го датасетот од линкот кој ви е поставен на самата задача, потоа истиот поставете го локално за да го вчитате.
Во случај на грешка при вчитување на .csv додадете аргумент `encoding='latin-1`


In [7]:
! pip install -U accelerate
! pip install -U transformers
! pip install transformers[torch] datasets evaluate

In [49]:
import pandas as pd
import numpy as np
import keras
from keras.layers import Input, Dense, LSTM, Embedding, concatenate
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential, Model
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

#Претпроцесирање кое е потребно

In [6]:
df=pd.read_csv("text_sentiment.csv")
df

text   Sentiment
0    Awe-inspired by the grandeur of an ancient cat...         Awe
1    Awe-struck by the grandeur of an ancient cathe...         Awe
2    Walking the Great Wall of China, each step a t...         Awe
3    Euphoria floods in as the final puzzle piece c...    Euphoria
4    Euphoria floods in as the final puzzle piece f...    Euphoria
..                                                 ...         ...
727  In the garden of friendships, each bloom a tes...  Friendship
728  A sense of wonder at the vastness of the cosmo...      Wonder
729  Awe-inspired by the vastness of the cosmos on ...      Wonder
730  Heartache deepens, a solitary journey through ...     Despair
731  In the carnival of emotions, the rollercoaster...      Thrill

[732 rows x 2 columns]

In [8]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()

In [10]:
df['Sentiment'] = encoder.fit_transform(df['Sentiment'])

In [11]:
df.head()

text  Sentiment
0  Awe-inspired by the grandeur of an ancient cat...         17
1  Awe-struck by the grandeur of an ancient cathe...         17
2  Walking the Great Wall of China, each step a t...         17
3  Euphoria floods in as the final puzzle piece c...         75
4  Euphoria floods in as the final puzzle piece f...         75

In [13]:
df.rename(columns={'Sentiment': 'label'}, inplace=True)

In [15]:
df

text  label
0    Awe-inspired by the grandeur of an ancient cat...     17
1    Awe-struck by the grandeur of an ancient cathe...     17
2    Walking the Great Wall of China, each step a t...     17
3    Euphoria floods in as the final puzzle piece c...     75
4    Euphoria floods in as the final puzzle piece f...     75
..                                                 ...    ...
727  In the garden of friendships, each bloom a tes...     84
728  A sense of wonder at the vastness of the cosmo...    187
729  Awe-inspired by the vastness of the cosmos on ...    187
730  Heartache deepens, a solitary journey through ...     50
731  In the carnival of emotions, the rollercoaster...    178

[732 rows x 2 columns]

In [36]:
X, Y = df['text'], df['label']

In [42]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, train_size=0.8,random_state=42)

In [40]:
train_data = pd.concat([X_train, Y_train], axis=1)

In [41]:
test_data = pd.concat([X_test, Y_test], axis=1)

In [59]:
test_data

text  label
13   Whispers of inspiration from the rustling leav...    108
390   Eyes wide open in the night, fearful shadows ...     80
708  Despite meticulous training, the swimmer faces...    130
602  At a Justin Bieber concert, the infectious bea...     71
47    Frustrated attempts to mend a broken connecti...     85
..                                                 ...    ...
625  Curiosity sparked by exploring a mysterious an...     46
55   Empowerment through mentoring and guiding others.     66
436  Imbued with gratitude for the simple pleasure ...     90
635  Shattered by betrayal, trust crumbles like fra...     19
48   Experiencing cyberbullying. Hateful messages o...     95

[147 rows x 2 columns]

In [60]:
train_data.to_csv('train.csv')
test_data.to_csv('test.csv')

In [64]:
pip install datasets --use-deprecated=legacy-resolver


Note: you may need to restart the kernel to use updated packages.


In [67]:
from datasets import load_dataset

dataset = load_dataset("csv", data_files={'train': 'train.csv', 'test': 'test.csv'})
dataset

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'text', 'label'],
        num_rows: 585
    })
    test: Dataset({
        features: ['Unnamed: 0', 'text', 'label'],
        num_rows: 147
    })
})

In [57]:
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

NameError: name 'tokenizer' is not defined

In [58]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("rizwan-ai/distilbert-base-uncased-finetuned-emotion")
tokenized_datasets = df.map(tokenize_function, batched=True)

tokenizer_config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

C:\Users\Davor\anaconda3\lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Davor\.cache\huggingface\hub\models--rizwan-ai--distilbert-base-uncased-finetuned-emotion. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

NameError: name 'dataset' is not defined

In [1]:
#Модел 1: Трансформер модел

In [39]:
pip install transformers 

Note: you may need to restart the kernel to use updated packages.


#Модел 2: Невронска мрежа

#Eвалуација на двата модели и споредба